In [33]:
import os
import sqlite3
import glob
import numpy as np


class HW3:

    def extract(self, input_files=None, out_db_filename="output.sqlite") -> None:
        connection = sqlite3.connect(out_db_filename)
        cursor = connection.cursor()

        cursor.execute('''
        CREATE TABLE IF NOT EXISTS output (it, v, epoch, U_beta_square_sum)
        ''')

        input_files = glob.glob("out*.txt")
        for file_name in input_files:
            lines = []
            with open(file_name, 'r') as reader:
                lines = reader.readlines()
                lines = [line.rstrip('\n') for line in lines]

            u_beta_lines = [
                line for line in lines
                if (line.startswith("it=") and "U_beta**2" in line)
            ]

            it: int = 0
            v: int = 0
            epoch: int = 0
            u: float = 0.0

            for line in u_beta_lines:
                it_part = line.find("it=")
                v_part = line.find("v=")
                epoch_part = line.find("epoch")
                u_beta_part = line.find("U_beta**2")

                it = int(line[it_part:v_part].split()[1])
                v = int(line[v_part:epoch_part].split()[1])
                epoch = int(line[epoch_part:u_beta_part].split()[1])

                u1 = line.find("sum is")
                u2 = line.find("change")
                u = float(line[(u1+len("sum is")):u2])

                cursor.execute('''
                INSERT INTO output (it, v, epoch, U_beta_square_sum)
                VALUES (?, ?, ?, ?)
                ''', (it, v, epoch, u))
        connection.commit()
        connection.close()


In [34]:
hw3 = HW3()
hw3.extract()